In [73]:
from stackapi import StackAPI
import pandas as pd
SITE = StackAPI('stackoverflow')

In [79]:
questions = SITE.fetch('questions', fromdate=1457136000, todate=1478222400,
                       min=20, sort='votes')
qdf = pd.DataFrame.from_dict(questions)
qdf = qdf.to_csv("questions.csv")

In [98]:
import ast
answerlist = []
qdf = pd.read_csv("questions.csv")
for index, row in qdf.iterrows():
    rd = ast.literal_eval(row['items'])
    if 'accepted_answer_id' in rd and 'user_id' in rd['owner']:
        answerlist.append(rd['accepted_answer_id'])

In [99]:
numiters = int(len(answerlist) / 99)
adf = {}
if numiters == 0:
    answers = SITE.fetch('/answers/{ids}', ids=answerlist)
else:
    answers = SITE.fetch('/answers/{ids}', ids=answerlist[:99])
    adf = pd.DataFrame.from_dict(answers)
    j = 99
    k = j + 99
    for i in range(0, numiters):
        answers = SITE.fetch('/answers/{ids}', ids=answerlist[j:k])
        temp = pd.DataFrame.from_dict(answers)
        adf = pd.concat([adf, temp])
        j = j + 99
        k = k + 99
adf = adf.reset_index()
adf = adf.to_csv("answers.csv")

In [109]:
#Only run above when needed.
import ast
import pandas as pd
userasklist = []
useranswerlist = []
questionlist = []
adf = pd.read_csv("answers.csv")
qdf = pd.read_csv("questions.csv")
dq = {}
for index, row in qdf.iterrows():
    q = ast.literal_eval(row['items'])
    if 'accepted_answer_id' in q and 'user_id' in q['owner']:
        dq[q['question_id']] = q['owner']['user_id']

In [110]:
da = {}
for index, row in adf.iterrows():
    q = ast.literal_eval(row['items'])
    if 'user_id' in q['owner']:
        da[q['question_id']] = q['owner']['user_id']
    else:
        #find a way to delete this
        da[q['question_id']] = "Null"

d = {"asked": pd.Series(dq.values(), index=dq.keys()), "answered": pd.Series(da.values(), index=dq.keys())}
data = pd.DataFrame(d)
data = data.reset_index()
data = data.rename(columns = {'index':'question'})
print data

     question answered    asked
0    38516736  2257100   722393
1    40282118  1774667   853791
2    38569992  6635732  4599914
3    36438034  1048572   596041
4    36597780  5968600  1219368
5    37124120  6312516  3364192
6    39506949    88656  3774685
7    37945376    95060    95060
8    37449761   209103  1174302
9    37548066   819742    88522
10   35987493    60281   964080
11   36872230   335858  5619103
12   39288891   415784  1027706
13   38713405  6052290  2629219
14   37812286  2227743  5393528
15   37800001   743382  6461611
16   38574148   459640  2817885
17   38870710  3758710  4507367
18   37904739   524555  5262010
19   37371990   209103  5582307
20   38453335   984780  5333340
21   36407386    17034  3980929
22   37843049  5644794  1082219
23   38813819  1433362  6687886
24   39215229  5818824   752630
25   40167559   246886  6794772
26   39471241  3032289  1082379
27   39399563  1002119  1002119
28   36759694  6170636  2032608
29   36411536  1177901  1177901
..      

In [106]:
from graphlab import SGraph, Vertex, Edge, canvas
canvas.set_target('ipynb')
graph = SGraph()
nodelist = []
for index, row in data.iterrows():
    nodelist.append(Vertex(row['asked']))
    nodelist.append(Vertex(row['answered']))
graph = graph.add_vertices(nodelist)
for index, row in data.iterrows():
    graph = graph.add_edges(Edge(row['asked'], row['answered']))
graph.show(vlabel = 'id')

KeyboardInterrupt: 

In [111]:
import networkx as nx

g = nx.from_pandas_dataframe(data, "asked", "answered")

In [112]:
#calcualte diameter.  We must pick a subset that is entirely connected and check that diameter.  
#We will use the biggest network
bn = sorted(nx.connected_components(g), key=len, reverse=True)[0]
g = g.subgraph(bn)
print nx.diameter(g)
print sorted(nx.degree_centrality(g).iteritems(), key=lambda(k, v): (-v, k))[0:9]

5
[(217408, 0.6153846153846154), (674039L, 0.23076923076923078), (1782465, 0.23076923076923078), (2069064L, 0.15384615384615385), (1401634L, 0.07692307692307693), (2622314L, 0.07692307692307693), (2822252L, 0.07692307692307693), (2951897L, 0.07692307692307693), (2998612L, 0.07692307692307693)]
